In [ ]:
# !pip install --upgrade git+https://github.com/flairNLP/flair.git
!pip install unidecode

In [ ]:
#Alternate way to connect google drive to colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Character embeddings saved dataset
# %time !cp "/content/drive/My Drive/Colab Notebooks/train.npz" "/content/train.npz" 
# %time !cp "/content/drive/My Drive/Colab Notebooks/val.npz" "/content/val.npz"

In [ ]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
# ! unzip crawl-300d-2M-subword.zip
from google.colab import files
uploaded = files.upload()

In [ ]:
! cp kaggle.json /root/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json
! kaggle datasets download -d peopledatalabssf/free-7-million-company-dataset
! unzip free-7-million-company-dataset.zip

In [ ]:
import pandas as pd
from collections import Counter, defaultdict
from tqdm import tqdm
import numpy as np
import re
from unidecode import unidecode
import string
import operator
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stopset = list(string.punctuation)
stopset.remove("&")

In [ ]:
df = pd.read_csv('companies_sorted.csv')
df.head()

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960


In [ ]:
print (df.shape, df.columns, df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7173426 entries, 0 to 7173425
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Unnamed: 0                 int64  
 1   name                       object 
 2   domain                     object 
 3   year founded               float64
 4   industry                   object 
 5   size range                 object 
 6   locality                   object 
 7   country                    object 
 8   linkedin url               object 
 9   current employee estimate  int64  
 10  total employee estimate    int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 602.0+ MB
(7173426, 11) Index(['Unnamed: 0', 'name', 'domain', 'year founded', 'industry',
       'size range', 'locality', 'country', 'linkedin url',
       'current employee estimate', 'total employee estimate'],
      dtype='object') None


In [ ]:
freq = Counter()
df["name"].dropna(inplace=True)
unique_companies = df["name"].unique()

for name in tqdm(unique_companies):
    freq.update(str(name).split(" "))

print(freq.most_common(30))
print (len(unique_companies))
key_words = [word for (word,_) in freq.most_common(30)]

In [ ]:
# from flair.data import Sentence
# from flair.embeddings import FastTextEmbeddings, TransformerWordEmbeddings, FlairEmbeddings, WordEmbeddings, StackedEmbeddings, BytePairEmbeddings

# # init glove embedding
# glove_embedding = WordEmbeddings('glove')

# # init fasttext embedding
# fast_embedding = FastTextEmbeddings('crawl-300d-2M-subword.bin')

# # init Flair embeddings
# flair_forward_embedding = FlairEmbeddings('multi-forward')
# flair_backward_embedding = FlairEmbeddings('multi-backward')

# # init multilingual BERT
# bert_embedding = TransformerWordEmbeddings('bert-base-multilingual-cased')

# # init embedding
# stack_embedding = StackedEmbeddings(
#     [
#         # standard FastText word embeddings for English
#         WordEmbeddings('en'),
#         # Byte pair embeddings for English
#         BytePairEmbeddings('en'),
#     ]
# )

In [ ]:
# def clean(word):
#     word = word.lower()
#     word = re.sub("[^A-Za-z0-9&]+", " ", word)
#     w_t = " ".join([i for i in word_tokenize(word) if i not in stopset])
#     w_t = unidecode(w_t)
#     return w_t

# def load_embeddings(embedding):
#     embed_dict = dict()
#     for i in tqdm(range(10000)):
#         token_emb = []
#         c = clean(unique_companies[i])
#         sentence = Sentence(c)
#         glove_embedding.embed(sentence)
#         for token in sentence:
#             token_emb.append(token.embedding.numpy())
#         embed_dict[unique_companies[i]] = np.array(token_emb).mean(axis=0).tolist()
#     return embed_dict

In [ ]:
# embed = load_embeddings(glove_embedding)
# print (len(embed))

In [ ]:
# def get_similarity(token, top_n=5):
#     res = dict()
#     c = clean(token)
#     sentence = Sentence(c)
#     glove_embedding.embed(sentence)
#     token_emb = []
#     for token in sentence:
#         token_emb.append(token.embedding.numpy())
#     target_emb = np.array(token_emb).mean(axis=0).tolist()

#     for k, v in embed.items():
#         emb_dist = np.linalg.norm(np.array(target_emb) - np.array(v))
#         res[k] = emb_dist

#     sorted_dict = sorted(res.items(), key=operator.itemgetter(1))
#     return sorted_dict[:top_n]

In [ ]:
# print (get_similarity("ibm limited"))
# print (get_similarity("e & y"))

### Two input siamese

Approach 1 : Using only character embeddings and Approach 2 : Bert Embeddings

In [ ]:
def clean(word):
    word = word.lower()
    word = re.sub("[^A-Za-z0-9&]+", " ", word)
    w_t = " ".join([i for i in word_tokenize(word) if i not in stopset])
    w_t = unidecode(w_t)
    return w_t

clean_names = []
for i in tqdm(range(len(unique_companies))):
    clean_names.append(clean(unique_companies[i]))

100%|██████████| 7004634/7004634 [08:35<00:00, 13593.98it/s]


In [ ]:
from collections import OrderedDict
from itertools import chain
import copy
import random
random.seed(42)

chars = sorted(list(OrderedDict.fromkeys(chain.from_iterable(clean_names))))
# remove empty space
chars.pop(0)
chars

['&',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
max_word_len = max([len(n) for n in clean_names])
max_word_len

108

In [ ]:
c2id = {c : i for i, c in enumerate(chars)}
id2c = {i : c for i, c in enumerate(chars)}
c2id

{'&': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36}

In [ ]:
# import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dropout, Dense, Flatten, Bidirectional, BatchNormalization
from keras.layers import Subtract, Multiply, Lambda, Concatenate, Input, GRU, GlobalMaxPool1D
from keras.models import Model
import keras.backend as K
import copy

Using TensorFlow backend.


In [ ]:
# create alias for given company names using different rules
# we need different ways one company name can be represented

def custom_rules(name):
    na = copy.deepcopy(name)
    if ((" & " in name) | (" and " in name) and (name.count(" & ") == 1 | name.count(" and ") == 1)):
        company_dict[na].append(name.replace(" & ", " and "))
        if (" and " in name):
            name = name.replace(" and ", " & ")
            company_dict[na].append(name)
        if (len(name.split(" & ")) == 1):
            name = name.replace("&", " & ")
        if (len(name.split(" & ")[0]) > 1):
            pre = ""
            try:
                for i in name.split(" & ")[0].split(" "):
                    if (len(i)!=0):
                        pre += i[0]
                        pre += " "
            except IndexError:
                pre = name.split(" & ")[0][0]
            company_dict[na].append(pre + "and " + name.split("& ")[1])
            company_dict[na].append(pre + "& " + name.split("&")[1])
        elif (len(name.split("&")[1]) > 1):
            company_dict[na].append(name.split(" & ")[0] + " and " + name.split(" & ")[1].split(" ")[0] + " " + (" ").join(name.split(" & ")[1].split(" ")[1:]))
            company_dict[na].append(name.split(" & ")[0] + " & " + name.split(" & ")[1].split(" ")[0] + " " + (" ").join(name.split(" & ")[1].split(" ")[1:]))
        elif (len(name.split(" & ")[0]) > 1 and len(name.split(" & ")[1]) > 1):
            pre = ""
            try:
                for i in name.split(" & ")[0].split(" "):
                    if (len(i)!=0):
                        pre += i[0]
                        pre += " "
            except IndexError:
                pre = name.split(" & ")[0][0]
            company_dict[na].append(pre + "and " + name.split(" & ")[1].split(" ")[0] + " " + (" ").join(name.split(" & ")[1].split(" ")[1:]))
            company_dict[na].append(pre + "& " + name.split(" & ")[1].split(" ")[0] + " " + (" ").join(name.split(" & ")[1].split(" ")[1:]))
    if ("inc" in name or "incorporated" in name):
        company_dict[na].append(name.replace("inc", "incorporation"))
    if ("pvt" in name):
        company_dict[na].append(name.replace("pvt", "private"))
    if ("private" in name):
        company_dict[na].append(name.replace("private", "pvt"))
    if ("company" in name):
        company_dict[na].append(name.replace("company", "co"))
    if ("ltd" in name):
        company_dict[na].append(name.replace("ltd", "limited"))
    if ("limited" in name):
        company_dict[na].append(name.replace("limited", "ltd"))


In [ ]:
company_dict = defaultdict(list)
for c in clean_names:
    company_dict[c].append(c)

In [ ]:
for c in tqdm(company_dict.keys()):
    custom_rules(c)

100%|██████████| 6968165/6968165 [00:12<00:00, 562412.30it/s]


In [ ]:
print ("Previous Length:", len(list(company_dict.keys())))
new_dict = defaultdict(list)
for k, v in company_dict.items():
    if len(list(set(v))) >= 2 and not k.startswith("&") and not k.startswith("and") and not k[0].isdigit() and len(k) >= 10:
        new_dict[k] = v
print ("New Length:", len(list(new_dict.keys())))

Previous Length: 6968165
New Length: 1467895




#### Approach 1 : Using character Embeddings

In [ ]:
split = int(0.8 * len(new_dict.keys()))
train, val = dict(list(new_dict.items())[:split]), dict(list(new_dict.items())[split:])
print ("Training:", len(list(train.keys())))
print ("Validation:", len(list(val.keys())))

In [ ]:
train = dict(sorted(train.items()))
val = dict(sorted(val.items()))

In [ ]:
class Dataset:
    def __init__(self, name_dict, c2id):
        self.name = name_dict
        self.c2id = c2id

    def __len__(self):
        return len(self.name.keys())

    def __getitem__(self, idx):
        X1 = []
        chosen = list(self.name.values())[idx]
        cname = random.choice(chosen).split(" ")
        cname = list(filter(None, cname))
        for word in cname:
            for char in word:
                X1.append(self.c2id[char])
        X1.extend([len(self.c2id)+1 for i in range(max_word_len-len(X1))])
        X2, Y = self._draw(cname, chosen, idx)
        return np.array(X1), np.array(X2), np.array(int(Y))
    
    def _draw(self, orig_word, chosen, chosen_idx):
        same = random.random() < 0.5
        # if not same, get the words having same first letter
        # print (same, orig_word, chosen_idx, chosen)
        if not same:
            x2 = []
            # candidates = [idx for idx, k in enumerate(self.name.keys()) if k[0] == orig_word[0][0]]
            candidates = []
            candidates = [idx for idx, k in enumerate(list(self.name.keys())[chosen_idx+1:chosen_idx+11]) if k[0] == orig_word[0][0]]
            candidates.extend([idx for idx, k in enumerate(list(self.name.keys())[chosen_idx-11:chosen_idx-1]) if k[0] == orig_word[0][0]])
            # print (candidates, orig_word, chosen_idx)
            cname = random.choice(list(self.name.values())[random.choice(candidates)]).split(" ")
            # print (cname)
            for word in cname:
                for char in word:
                    x2.append(self.c2id[char])
            x2.extend([len(self.c2id)+1 for i in range(max_word_len-len(x2))])
        # choose a different word other than the chosen original word
        else:
            x2 = []
            while (1):
                cname = random.choice(chosen).split(" ")
                if orig_word != cname:
                    break
            for word in cname:
                for char in word:
                    x2.append(self.c2id[char])
            x2.extend([len(self.c2id)+1 for i in range(max_word_len-len(x2))])
        return x2, same

# class Dataloader(keras.utils.Sequence):
#     def __init__(self, dataset, batch_size: int = 256):
#         self.dataset = dataset
#         self.batch_size = batch_size

#     def __len__(self):
#         return len(self.dataset) // self.batch_size

#     def __getitem__(self, i):
#         start = i * self.batch_size
#         stop = (i + 1) * self.batch_size
#         data = []
#         for j in range(start, stop):
#             d = self.dataset[j]
#             data.append([[d[0], d[1]], d[2]])

#         # transpose list of lists
#         batch = [np.stack(samples, axis=0) for samples in zip(*data)]
#         return batch

In [ ]:
train_dataset = Dataset(train, c2id)
val_dataset = Dataset(val, c2id)

In [ ]:
x, y, z = val_dataset[0]
c1 = ""
for n in x:
    if n != 38:
        c1 += id2c[n]
    else:
        break
c2 = ""
for n in y:
    if n != 38:
        c2 += id2c[n]
    else:
        break
print (c1, c2, z, x.shape, y.shape)

a&aadviesgroep aandabuildingservices 1 (108,) (108,)


In [ ]:
# # train_dataloader = Dataloader(train_dataset)
# # val_dataloader = Dataloader(val_dataset)
# # len(train_dataloader),len(train_dataset),

# %%time

# train_x1 = []
# train_x2 = []
# train_y = []
# for i in range(100000):
#     data = train_dataset[i]
#     train_x1.append(data[0])
#     train_x2.append(data[1])
#     train_y.append(data[2])

# val_x1 = []
# val_x2 = []
# val_y = []
# for i in range(5000):
#     data = val_dataset[i]
#     val_x1.append(data[0])
#     val_x2.append(data[1])
#     val_y.append(data[2])

CPU times: user 3min 17s, sys: 5.34 ms, total: 3min 17s
Wall time: 3min 17s


In [ ]:
# np.savez('train.npz', name1=np.array(train_x1), name2=np.array(train_x2), name3=np.array(train_y))
# np.savez('val.npz', name1=np.array(val_x1), name2=np.array(val_x2), name3=np.array(val_y))

# %time !cp "/content/train.npz" "/content/drive/My Drive/Colab Notebooks/train.npz"
# %time !cp "/content/val.npz" "/content/drive/My Drive/Colab Notebooks/val.npz"

In [ ]:
data = np.load("train.npz")
train_x1, train_x2, train_y = data["name1"], data["name2"], data["name3"] 
data = np.load("val.npz")
val_x1, val_x2, val_y = data["name1"], data["name2"], data["name3"] 

In [ ]:
# train_x1, train_x2, train_y = np.array(train_x1), np.array(train_x2), np.array(train_y)
# val_x1, val_x2, val_y = np.array(val_x1), np.array(val_x2), np.array(val_y)

idx = np.arange(train_x1.shape[0])
random.shuffle(idx)
train_x1, train_x2, train_y = train_x1[idx, :], train_x2[idx, :], train_y[idx]

In [ ]:
def cosine_distance(vects):
    x, y = vects
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

thresh = 0.5
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < thresh, y_true.dtype)))

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return 0.5 * K.mean(y_true * square_pred + (1-y_true) * margin_square)

def siamese_network():
    inp = Input(shape=(max_word_len,))
    x = Embedding(input_dim=len(c2id.keys())+1,
                  output_dim=128,
                  input_length=max_word_len)(inp)
    x = GRU(128, return_sequences=True)(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(x)
    return Model(inputs=inp, outputs=norm_layer)

siamese_model = siamese_network()
siamese_model.summary()

Model: "model_80"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_117 (InputLayer)       (None, 108)               0         
_________________________________________________________________
embedding_43 (Embedding)     (None, 108, 128)          4864      
_________________________________________________________________
gru_45 (GRU)                 (None, 108, 128)          98688     
_________________________________________________________________
flatten_40 (Flatten)         (None, 13824)             0         
_________________________________________________________________
dense_81 (Dense)             (None, 64)                884800    
_________________________________________________________________
dropout_39 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_82 (Dense)             (None, 32)                208

In [ ]:
i1_inp = Input(shape=(max_word_len,))
i2_inp = Input(shape=(max_word_len,))

i1_embed = siamese_model(i1_inp)
i2_embed = siamese_model(i2_inp)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([i1_embed, i2_embed])

model = Model(inputs=[i1_inp, i2_inp], outputs=distance)
model.compile(loss=contrastive_loss, optimizer='adam', metrics=[accuracy])
print (model.summary())

Model: "model_81"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_118 (InputLayer)          (None, 108)          0                                            
__________________________________________________________________________________________________
input_119 (InputLayer)          (None, 108)          0                                            
__________________________________________________________________________________________________
model_80 (Model)                (None, 32)           990432      input_118[0][0]                  
                                                                 input_119[0][0]                  
__________________________________________________________________________________________________
lambda_39 (Lambda)              (None, 1)            0           model_80[1][0]            

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True, verbose=1)]
history = model.fit(x=[train_x1, train_x2], y=train_y, 
                    batch_size=1024, 
                    epochs=5,
                    callbacks=callbacks,
                    validation_data=[[val_x1, val_x2], val_y])

In [ ]:
def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return np.mean(labels==(predictions.ravel() < thresh))

preds = model.predict(x=[val_x1, val_x2])
print (compute_accuracy(preds, val_y))

0.661


In [ ]:
for i in range(50):
    c1 = ""
    for n in val_x1[i]:
        if n != 38:
            c1 += id2c[n]
    c2 = ""
    for n in val_x2[i]:
        if n != 38:
            c2 += id2c[n]
    print (c1, c2, preds[i], "G:", val_y[i], "P:", int(preds[i] < thresh))

a&aadviesgroep aandadisposal [1.052872] G: 0 P: 0
a&aaerospaceinc aandaaerospaceinc [0.35893422] G: 1 P: 1
a&aarchitectssdnbhd a&aadviesgroep [0.8297443] G: 0 P: 0
aandabeautysalonltd aandaarchitectssdnbhd [0.26824713] G: 0 P: 1
a&abuildingservices aandabuildingservices [0.95419043] G: 1 P: 0
aandaconstructioncompany a&abeautysalonlimited [0.00031623] G: 0 P: 1
a&acourtreportingandvideoconferencing a&acourtreporting [0.0337266] G: 1 P: 1
a&adentallabltd aandabuildingservices [0.95419043] G: 0 P: 0
a&adisposal aandadisposal [0.4216389] G: 1 P: 1
aandadrywallllc aandadrywallllc [0.00031623] G: 0 P: 1
a&aelectronicsassemblyinc aandaelectronicsassemblyinc [0.00031623] G: 1 P: 1
a&aengcoltdacehumanism&acebusiness aandaengcoltdacehumanism [0.00031623] G: 1 P: 1
a&aenterprisesparentcompanyofblueprintacademy&innovativeonlinesolutions aandabeautysalonltd [0.00031623] G: 0 P: 1
a&aevents&marketing a&adrywallllc [0.00031623] G: 0 P: 1
aandafacilityservicesinc aandabuildingservices [0.63422334] G:

In [ ]:
model.save("siamese_company.h5")
%time !cp "/content/siamese_company.h5" "/content/drive/My Drive/Colab Notebooks/siamese_company.h5"

#### Approach 2 : Using BERT embeddings

In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 40kB 3.5MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=8b6598e76a1ee7d42a8790735c1a2592276b798da709e341e06453fc7cc6f79a
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=12d3ea3c3973d115fe17e838367348feb10815e726c7e982069e180917d9b566
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=f10f9ae9a6c27d4e4d20b0da29b16ed51c8cad88b0d7cfc92691ede9357cd975
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 6.7MB/s 


In [ ]:
split = int(0.8 * len(new_dict.keys()))
train, val = dict(list(new_dict.items())[:split]), dict(list(new_dict.items())[split:])
print ("Training:", len(list(train.keys())))
print ("Validation:", len(list(val.keys())))

Training: 1174316
Validation: 293579


In [ ]:
train = dict(sorted(train.items()))
val = dict(sorted(val.items()))

In [ ]:
import keras
from keras.preprocessing import sequence
from keras.layers import Embedding, LSTM, Dropout, Dense, Flatten, Bidirectional, BatchNormalization
from keras.layers import Subtract, Multiply, Lambda, Concatenate, Input, GRU, GlobalMaxPool1D, Conv1D
from keras.models import Model
import keras.backend as K
import bert
import tensorflow_hub as hub
from bert import bert_tokenization
import random
random.seed(42)

max_word_len = 108
BertTokenizer = bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
class Dataset:
    def __init__(self, name_dict):
        self.name = name_dict

    def __len__(self):
        return len(self.name.keys())

    def __getitem__(self, idx):
        chosen = list(self.name.values())[idx]
        cname = random.choice(chosen)
        token = tokenizer.tokenize(cname)
        X1 = tokenizer.convert_tokens_to_ids(token)
        X1.extend([0 for i in range(max_word_len-len(X1))])
        # print (cname, token, X1)
        X2, Y = self._draw(cname, chosen, idx)
        return np.array(X1), np.array(X2), np.array(int(Y))
    
    def _draw(self, orig_word, chosen, chosen_idx):
        same = random.random() < 0.5
        # if not same, get the words having same first letter
        # print (same, orig_word, chosen_idx, chosen)
        if not same:
            x2 = []
            # candidates = [idx for idx, k in enumerate(self.name.keys()) if k[0] == orig_word[0][0]]
            candidates = []
            candidates = [idx for idx, k in enumerate(list(self.name.keys())[chosen_idx+1:chosen_idx+11]) if k[0] == orig_word[0][0]]
            candidates.extend([idx for idx, k in enumerate(list(self.name.keys())[chosen_idx-11:chosen_idx-1]) if k[0] == orig_word[0][0]])
            # print (candidates, orig_word, chosen_idx)
            cname = random.choice(list(self.name.values())[random.choice(candidates)])
            token = tokenizer.tokenize(cname)
            x2 = tokenizer.convert_tokens_to_ids(token)
            x2.extend([0 for i in range(max_word_len-len(x2))])
            # print (cname, token, x2)
        # choose a different word other than the chosen original word
        else:
            x2 = []
            while (1):
                cname = random.choice(chosen)
                if orig_word != cname:
                    break
            token = tokenizer.tokenize(cname)
            x2 = tokenizer.convert_tokens_to_ids(token)
            x2.extend([0 for i in range(max_word_len-len(x2))])
            # print (cname, token, x2)
        return x2, same


In [ ]:
train_dataset = Dataset(train)
val_dataset = Dataset(val)

In [ ]:
x, y, z = train_dataset[0]
print (x, y, z)
c1 = tokenizer.convert_ids_to_tokens(x)
c2 = tokenizer.convert_ids_to_tokens(y)
print (c1, c2, z, x.shape, y.shape)

[ 1037  1004  1037  3229 10050  4371 26189  2102  3132     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0] [ 1037  1998  1037  3229 10050  4371 26189  2102  5183     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0  

In [ ]:
%%time

train_x1 = []
train_x2 = []
train_y = []
for i in range(100000):
    data = train_dataset[i]
    train_x1.append(data[0])
    train_x2.append(data[1])
    train_y.append(data[2])

val_x1 = []
val_x2 = []
val_y = []
for i in range(5000):
    data = val_dataset[i]
    val_x1.append(data[0])
    val_x2.append(data[1])
    val_y.append(data[2])

CPU times: user 5h 36min 34s, sys: 28.8 s, total: 5h 37min 3s
Wall time: 5h 37min 1s


In [ ]:
np.savez('train_bert.npz', name1=np.array(train_x1), name2=np.array(train_x2), name3=np.array(train_y))
np.savez('val_bert.npz', name1=np.array(val_x1), name2=np.array(val_x2), name3=np.array(val_y))

%time !cp "/content/train_bert.npz" "/content/drive/My Drive/Colab Notebooks/train_bert.npz"
%time !cp "/content/val_bert.npz" "/content/drive/My Drive/Colab Notebooks/val_bert.npz"

CPU times: user 48.5 ms, sys: 56 ms, total: 104 ms
Wall time: 15.3 s
CPU times: user 44.7 ms, sys: 59 ms, total: 104 ms
Wall time: 15.3 s


In [ ]:
data = np.load("train_bert.npz")
train_x1, train_x2, train_y = data["name1"], data["name2"], data["name3"] 
data = np.load("val_bert.npz")
val_x1, val_x2, val_y = data["name1"], data["name2"], data["name3"] 

In [ ]:
train_x1, train_x2, train_y = np.array(train_x1), np.array(train_x2), np.array(train_y)
val_x1, val_x2, val_y = np.array(val_x1), np.array(val_x2), np.array(val_y)

idx = np.arange(train_x1.shape[0])
random.shuffle(idx)
train_x1, train_x2, train_y = train_x1[idx, :], train_x2[idx, :], train_y[idx]

In [ ]:
thresh = 0.5

def cosine_distance(vects):
    x, y = vects
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < thresh, y_true.dtype)))

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return 0.5 * K.mean(y_true * square_pred + (1-y_true) * margin_square)

# def siamese_network():
#     inp = Input(shape=(max_word_len,))
#     x = Embedding(input_dim=len(tokenizer.vocab)+1,
#                   output_dim=128,
#                   input_length=max_word_len)(inp)
#     x = GRU(128, return_sequences=True)(x)
#     x = Flatten()(x)
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.4)(x)
#     x = Dense(32, activation='relu')(x)
#     norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(x)
#     return Model(inputs=inp, outputs=norm_layer)

def siamese_network():
    inp = Input(shape=(max_word_len,))
    x = Embedding(input_dim=len(tokenizer.vocab)+1,
                  output_dim=200,
                  input_length=max_word_len)(inp)
    x = Conv1D(filters=100,
               kernel_size=3,
               padding="valid",
               activation="relu")(x)
    x = Conv1D(filters=100,
               kernel_size=4,
               padding="valid",
               activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    norm_layer = Lambda(lambda  x: K.l2_normalize(x, axis=1), name='norm_layer')(x)
    return Model(inputs=inp, outputs=norm_layer)

siamese_model = siamese_network()
siamese_model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 108)               0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 108, 200)          6104600   
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 106, 100)          60100     
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 103, 100)          40100     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 100)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0  

In [ ]:
i1_inp = Input(shape=(max_word_len,))
i2_inp = Input(shape=(max_word_len,))

i1_embed = siamese_model(i1_inp)
i2_embed = siamese_model(i2_inp)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([i1_embed, i2_embed])

model = Model(inputs=[i1_inp, i2_inp], outputs=distance)
model.compile(loss=contrastive_loss, optimizer='adam', metrics=[accuracy])
print (model.summary())

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           (None, 108)          0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           (None, 108)          0                                            
__________________________________________________________________________________________________
model_18 (Model)                (None, 32)           6213344     input_29[0][0]                   
                                                                 input_30[0][0]                   
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 1)            0           model_18[1][0]            

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True, verbose=1)]
history = model.fit(x=[train_x1, train_x2], y=train_y, 
                    batch_size=1024, 
                    epochs=5,
                    callbacks=callbacks,
                    validation_data=[[val_x1, val_x2], val_y])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 100000 samples, validate on 5000 samples
Epoch 1/5
100000/100000 [==============================] - 9s 93us/step - loss: 0.0208 - accuracy: 0.9469 - val_loss: 0.1000 - val_accuracy: 0.6826
Epoch 2/5
100000/100000 [==============================] - 8s 80us/step - loss: 6.0697e-05 - accuracy: 0.9999 - val_loss: 0.1887 - val_accuracy: 0.5637
Epoch 3/5
100000/100000 [==============================] - 8s 80us/step - loss: 3.7642e-05 - accuracy: 0.9999 - val_loss: 0.1755 - val_accuracy: 0.5637
Restoring model weights from the end of the best epoch
Epoch 00003: early stopping


In [ ]:
def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return np.mean(labels==(predictions.ravel() < thresh))

preds = model.predict(x=[val_x1, val_x2])
print (compute_accuracy(preds, val_y))

0.683


In [ ]:
model.save("siamese_company_bert.h5")
%time !cp "/content/siamese_company_bert.h5" "/content/drive/My Drive/Colab Notebooks/siamese_company_bert.h5"

CPU times: user 9.76 ms, sys: 65.1 ms, total: 74.9 ms
Wall time: 2.24 s


In [ ]:
print (np.unique(val_y, return_counts=True))

(array([0, 1]), array([2452, 2548]))
